# Learning how to Linear Regression on a housing dataset
Follow allong with the tutorial [Sklearn learning model for beginner](https://youtu.be/-IvNzmrcyUM?si=yFj89wcu9vC6wDk9) on Youtube.

In [1]:
%pip install scikit-learn joblib

Note: you may need to restart the kernel to use updated packages.


## Import dataset
Import the dataset from the default datasets of sklearn, using return_X_y flag to get a tuple.

In [2]:
from sklearn import datasets

X, y = datasets.fetch_california_housing(return_X_y=True)
print(X,y)

[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]] [4.526 3.585 3.521 ... 0.923 0.847 0.894]


## Splitting the dataset
In order to train the model we need to split the dataset into a 80-20 percent for training and testing data.

This is acheived with the `train_test_split` function and the `test_size=0.2` parameter.

The `random_state=432` parameter allow us to shuffle the samples of data in a specific order instead of randomly (by default). Hence we can reproduce the exact workflow, it's like a seed in stable diffusion.

In [3]:
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=432)
print(f"X 80% Training Data: {X_train}, {y_train}")
print(f"Y 20% Testing Data: {x_test}, {y_test}")


X 80% Training Data: [[   2.1442       52.            3.94886364 ...    2.61647727
    37.34       -121.88      ]
 [   4.0938       20.            5.66       ...    2.668
    38.35       -121.96      ]
 [   5.3925       51.            5.00677966 ...    2.54237288
    34.2        -118.23      ]
 ...
 [   4.3958       10.            6.15450644 ...    3.78540773
    34.09       -117.39      ]
 [   2.8125       29.            4.33367983 ...    2.1039501
    37.72       -122.15      ]
 [   3.4934       27.            5.46       ...    3.00666667
    37.57       -120.85      ]], [1.889 1.173 3.179 ... 1.307 1.574 1.938]
Y 20% Testing Data: [[   5.1039       52.            5.62803738 ...    2.43551402
    37.76       -122.23      ]
 [   2.5238       49.            4.19202363 ...    2.19645495
    37.8        -122.24      ]
 [   5.7393        9.            5.79227053 ...    3.03864734
    34.03       -117.31      ]
 ...
 [   3.5588       37.            4.26751592 ...    1.97770701
    37.97   

## Training model
We'll use [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) to guess houses prices based on our dataset.

> Ordinary least squares Linear Regression.
> 
> LinearRegression fits a linear model with coefficients w = (w1, …, wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

### Description
The model will go sample by sample to compare each set of feature (X_train) to it's target (y_train).
This will make the model trained over our training data, but we want it to be able to predict the house price outside of our training data.

In [4]:
from sklearn.linear_model import LinearRegression

model = LinearRegression() # Model instanciation
model.fit(X_train, y_train) # Training the model

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


## Evaluating the model
We want to be sure the model learned data instead of memorizing it (overfitting).
For that we'll evaluate the model with the `x_test` feature part in order to have the `y_prediction`; once we have the `y_prediction`, we'll be able to compare top the values in `y_test`.

For the evaluation we use [r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) which is a regression score function (best possible is 1.0)

In [5]:
from sklearn.metrics import r2_score

y_prediction = model.predict(x_test)
r2 = r2_score(y_test, y_prediction) # Comparing the actual value to the y_test

print(r2)

# The accuracy score is 0.6080229586580346 so the model understand 60% of why the house price goes up and down.
# This is a baseline to improve it further

0.6080229586580346


## Polynomial Features
A Polynomial feature allow to increase features in a regression.

With sklearn PolynomialFeatures, we can take our 8 -> 45 features and r2 score from 0.6 to 0.66 (0.2 increase).

### 📊 Useful
- In polynomial regression, applying a linear regression on polynomial features enable us to get the non linear relations.
- Enrich the features when you think interactions are important (ex: age x income).

### ⚠️ Warning
- The more you increase the `degree` parameter in `PolynomialFeatures()` the mode you have new columns.
- Can lead to overfitting if dataset is small.

In [6]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures()

print(f"Before Polynomial transformation {X.shape}")
X_poly = poly.fit_transform(X.data)
print(f"After Polynomial transformation: {X_poly.shape}")

Before Polynomial transformation (20640, 8)
After Polynomial transformation: (20640, 45)


## Selecting the right model
The `HistGradientBoostingRegressor` and `RandomForestRegressor` models are good concurrent models and we'll try them with a regression score too.

In a loop of `[model, GBR, RFR]`:
1. we train the model on training data `X_train` and `y_train`
2. make prediction with `x_test`
3. score the regression with `y_test` and `y_pred`

This will return us multiple scores for multiple models.
```
Score for LinearRegression() is: 0.6080229586580346
Score for HistGradientBoostingRegressor() is: 0.8364438492135174
Score for RandomForestRegressor(n_jobs=-1) is: 0.8126642078111718
```

### Parameters
- `n_jobs=-1` is to use all cores available for the job, we can limit in positive numbers (1,3,10, etc)

In [7]:
from sklearn.ensemble import (HistGradientBoostingRegressor, RandomForestRegressor)

# Use HistGradientBoostingRegressor over GradientBoostingRegressor because it's an optimized version
GBR = HistGradientBoostingRegressor()
RFR = RandomForestRegressor(n_jobs=-1)

for i in [model, GBR, RFR]:
  i.fit(X_train, y_train)
  y_pred = i.predict(x_test)
  r2 = r2_score(y_test, y_pred)
  print(f"Score for {i} is: {r2}")

Score for LinearRegression() is: 0.6080229586580346
Score for HistGradientBoostingRegressor() is: 0.8343591831583239
Score for RandomForestRegressor(n_jobs=-1) is: 0.8162925735108956


## HyperParamertization : Parameter optimization
Now we'll evaluate what are the best parameters to use for our model.
How to do optimization ?
- First we experiment with the number of trees which is the `max_iter` parameter. In the `max_iteration_range` list we test every number.
- Then we search the learning rate

Results are
```
Starting Parameter optimization
Score for iteration of 250 with 0.1 is: 0.8434126795758516
Score for iteration of 300 with 0.1 is: 0.8426676580156846
Score for iteration of 350 with 0.1 is: 0.8439658387680059
Score for iteration of 250 with 0.05 is: 0.8375863801259467
Score for iteration of 300 with 0.05 is: 0.8411898616012664
Score for iteration of 350 with 0.05 is: 0.8448662786329125
Score for iteration of 250 with 0.001 is: 0.2608375545135623
Score for iteration of 300 with 0.001 is: 0.299112357206882
Score for iteration of 350 with 0.001 is: 0.33695141665669903
```
Which make the score for iteration of 350 with 0.05 the best at: 0.8448662786329125

In [ ]:
from sklearn.ensemble import (HistGradientBoostingRegressor)
max_iteration_range: list[int] = [250, 300, 350]
max_learning_rate: list[float] = [0.1, 0.05, 0.001]

def train_model(iteration, learning_rate) -> HistGradientBoostingRegressor:
  """
  This function is training model with max_item
  """
  model = HistGradientBoostingRegressor(max_iter=iteration, learning_rate=learning_rate)
  model.fit(X_train, y_train)
  return model

def evaluate_model(model, y_test, x_test) -> float:
  y_pred = model.predict(x_test)
  r2 = r2_score(y_test, y_pred)
  return r2

print("Starting Parameter optimization")
for j in max_learning_rate:
  for i in max_iteration_range:
    model = train_model(i, j)
    r2 = evaluate_model(model, y_test, x_test)
    print(f"Score for iteration of {i} with {j} is: {r2}")

Starting Parameter optimization
Score for iteration of 250 with 0.1 is: 0.8477969889764889
Score for iteration of 300 with 0.1 is: 0.8417915594601302
Score for iteration of 350 with 0.1 is: 0.8402866632786614
Score for iteration of 250 with 0.05 is: 0.8403650622621107
Score for iteration of 300 with 0.05 is: 0.8440546801360664
Score for iteration of 350 with 0.05 is: 0.843186462051422
Score for iteration of 250 with 0.001 is: 0.2591126203803521
Score for iteration of 300 with 0.001 is: 0.300888661180683
Score for iteration of 350 with 0.001 is: 0.3354502938206909


## Exporting model
Now we have our perfect parameters, we want to export our model in order to save it on the file system, thanks to [joblib](https://joblib.readthedocs.io/en/stable/persistence.html#use-case).

In [ ]:
import joblib

# Create final model with optimal parameters
HGBR_model = HistGradientBoostingRegressor(max_iter=350, learning_rate=0.05)
HGBR_model.fit(X_train, y_train)

joblib.dump(HGBR_model, "my_model.joblib")

print("Final model registered")

Final model registered


## Testing created model
Now that our model has been trained and exported we can use it to predict on data.

The result should be:
`The score of the trained model is: 0.8436238636435226`

In [14]:
trained_model = joblib.load("my_model.joblib")
y_predict = trained_model.predict(x_test)
r2 = r2_score(y_test, y_predict)
print(f"The score of the trained model is: {r2}")

The score of the trained model is: 0.8436238636435226
